# Beginning of Project

In [1]:
import os
import zipfile
import pandas as pd
import pyarrow as pa
import pyarrow.csv
import datetime as dt

In [2]:
non_state_codes = [
    "DC", "PR", "AE", "VI", "AP", "GU", 
    "MP", "AA", "AS", "EN", "GE", "QC", 
    "XX", "NO", "AB", "ZZ"
]
state_codes = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

In [3]:
# Open the merged ZIP file and extract the CSV
output_file = "P00000001-ALL.zip"
with zipfile.ZipFile(output_file, "r") as zip_ref:
    csv_filename = "P00000001-ALL.csv"
    csv_data = zip_ref.read(csv_filename)#.decode("utf-8")  # Read CSV as text

print(f"Extracted CSV: {csv_filename}")
cf_df = pa.csv.read_csv(pa.BufferReader(csv_data)).to_pandas()
#cf_df.iloc[:5]

Extracted CSV: P00000001-ALL.csv


In [4]:
# clean dataset 
# remove non state codes
cf_df = cf_df.loc[cf_df["contbr_st"].isin(state_codes)]
# change type of contb_receipt_dt column
cf_df["contb_receipt_dt"] = pd.to_datetime(cf_df["contb_receipt_dt"], format="%d-%b-%y")

In [5]:
x = dict(cf_df["contbr_st"].value_counts())
x = sorted(x.items(), key=lambda x: x[1], reverse=True)
x

[('CA', 2315984),
 ('TX', 1326243),
 ('FL', 1291672),
 ('NY', 900787),
 ('WA', 575317),
 ('PA', 565513),
 ('IL', 510471),
 ('VA', 488029),
 ('NC', 477878),
 ('AZ', 464248),
 ('GA', 451714),
 ('OH', 436920),
 ('MA', 435617),
 ('MI', 428944),
 ('NJ', 420016),
 ('CO', 383827),
 ('MD', 345356),
 ('OR', 324955),
 ('TN', 307066),
 ('MN', 284656),
 ('SC', 264966),
 ('MO', 252251),
 ('IN', 239801),
 ('WI', 228696),
 ('CT', 190887),
 ('AL', 179589),
 ('NV', 179087),
 ('OK', 168439),
 ('LA', 157091),
 ('UT', 147325),
 ('NM', 133366),
 ('KY', 126308),
 ('KS', 122317),
 ('IA', 113651),
 ('AR', 111944),
 ('ID', 111347),
 ('NH', 105191),
 ('ME', 90648),
 ('MS', 86034),
 ('NE', 73413),
 ('MT', 71174),
 ('HI', 68262),
 ('DE', 61585),
 ('WV', 55229),
 ('AK', 53548),
 ('RI', 48420),
 ('VT', 46883),
 ('WY', 38456),
 ('SD', 36873),
 ('ND', 29511)]

In [6]:
new = cf_df.set_index(["contbr_st","contbr_zip"])
new.head()

cmte_id    cand_id            cand_nm  \
contbr_st contbr_zip                                            
AK        99567       C00540302  P40012155  Scott, Timothy E.   
          99518       C00540302  P40012155  Scott, Timothy E.   
          99611       C00540302  P40012155  Scott, Timothy E.   
          99611       C00540302  P40012155  Scott, Timothy E.   
          99611       C00540302  P40012155  Scott, Timothy E.   

                      contb_receipt_amt contb_receipt_dt election_tp  
contbr_st contbr_zip                                                  
AK        99567                   100.0       2023-06-28       P2024  
          99518                    30.0       2023-06-25       P2024  
          99611                    50.0       2023-04-04       P2028  
          99611                   -50.0       2023-04-12       P2028  
          99611                    50.0       2023-04-12       P2024

In [7]:
new_summed = pd.DataFrame(new.groupby(level=["contbr_st", "contbr_zip"])["contb_receipt_amt"].sum())
new_summed.head()

contb_receipt_amt
contbr_st contbr_zip                   
AK        00000                  297.00
          01455                  260.04
          03281                 1206.50
          06066                  -25.00
          11111                  278.30

In [12]:
elec_df = pd.read_excel("2024presgeresults.xlsx")

# clean dataset
elec_df.fillna(0, inplace=True)

elec_df.head()

,STATE,ELECTORAL VOTES,ELECTORAL VOTE: TRUMP (R),ELECTORAL VOTE: HARRIS (D),AYYADURAI,BOWMAN,DE LA CRUZ,DUNCAN,EBKE,EVERYLOVE,...,STODDEN,SUPREME,TERRY,TRUMP,WELLS,WEST,WOOD,NONE OF THESE CANDIDATES,WRITE-INS (SCATTERED),TOTAL VOTES
0,AL,9,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1462616.0,0.0,0.0,0.0,0.0,8738.0,2265090.0
1,AK,3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,812.0,184458.0,0.0,1127.0,0.0,0.0,0.0,338177.0
2,AZ,11,11.0,0.0,77.0,0.0,689.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1770242.0,0.0,0.0,0.0,0.0,23.0,3390161.0
3,AR,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,759241.0,0.0,0.0,1144.0,0.0,0.0,1182676.0
4,CA,54,0.0,54.0,0.0,0.0,72539.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6081697.0,0.0,0.0,0.0,0.0,0.0,15865475.0


In [ ]:
# filter cf_df to only presidential candidates
